In [1]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from transformers import AutoTokenizer
from model.model.personalized_bert import EventBertModel
from model.model.PairwiseLecardPLM1 import PairwisePLM1
from formatter.PairwiseFormatter1 import PairwiseFormatter
from config_parser import create_config
from tools.init_tool1 import init_formatter
from torch.autograd import Variable

# plm_path = "/home/pub/fan/SCR/Pretrain_model/bert_base_chinese"
# tokenizer = AutoTokenizer.from_pretrained(plm_path)
device = torch.device("cuda")

/home/nlpc/miniconda3/envs/LED/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('/hhd2/fan/SCR/SCR-Experiment/utils/extracted_cases.csv')
texts = df['text'].tolist()
labels = df['label'].tolist()
def correct_state_dict(state_dict):
    """调整状态字典中的键名，以适应当前模型结构"""
    new_state_dict = {}
    for k, v in state_dict.items():
        # 移除所有不必要的'module.'前缀
        new_key = k.replace("module.", "")
        new_state_dict[new_key] = v
    return new_state_dict

def load_model(model_path):
    checkpoint = torch.load(model_path, map_location='cpu')
    model = PairwisePLM1()
    # 使用修正函数处理状态字典
    fixed_state_dict = correct_state_dict(checkpoint['model'])
    model.load_state_dict(fixed_state_dict)
    
    return model

# 假设你的模型保存路径为
model_path = '/hhd2/fan/SCR/output/3.pth'
model = load_model(model_path)

collate_fn = {}
formatter = {}
acc_result = None
data_path = "/hhd2/fan/SCR/SCR-Experiment/test.json"
def get_data(task_list, *args, **params):
    for task in task_list:
        print(task)
        return PairwiseFormatter(task, *args, **params).process_single(data_path, "test")
data = get_data(["test"], data_path)
print(data)
print(data["inputx"].shape)
# print(data.type)
with torch.no_grad():
    logits = model(data, "valid", acc_result)
    probs = torch.softmax(logits, dim=1)
    


using EDBERT (Event Detection BERT)


Some weights of the model checkpoint at /hhd2/fan/SCR/Pretrain_model/bert_base_chinese were not used when initializing EventBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing EventBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EventBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of EventBertModel were not initialized from the model checkpoint at /hhd2/fan/SCR/Pretrain_model/b

test
{'inputx': tensor([[  101,  3125,  2692,   839,  2154,  5389,  8138,  2399,   129,  3299,
          6629,  6158,  1440,   782,  7357,  2456,  1092,   680,  1453,  3378,
          5310,  6399,  1400,  2458,  1993,   769,  2518,  8024,  2400,   680,
          1453,  3378,  4638,  2207,  1992,  1987,  1155,  3378,  4508,  4225,
          2634,   511,  8127,  2399,  8108,  3299,  8128,  3189,  3241,  8024,
          1155,  3378,  4508,  6913,  6435,  1453,  3378,  5635,  3736,  7346,
          2356,  3378,  3378,  7252,  3378,  3378,   100,  1548,  3625,  8024,
          6158,  1440,   782,  7357,  2456,  1092,   510,  1155,  3378,  4508,
          4638,   675,  1923,  3330,  3378,  1469,  3301,  1351,  5918,  3378,
           510,  4374,  3378,  1044,  1400,  3341,  1168,   677,  6835,   102,
          2528,  8024,  1587,  8024,  3719,  8024,   510,  8024,  2528,  8024,
          4195,  8024,  3125,  8024,  2692,  8024,   839,  8024,  2154,  8024,
           671,  8024,  3428,  6629,

In [3]:
# test_dataset = init_test_dataset()

# for step, data in enumerate(test_dataset):
#     for key in data.keys():
#         if isinstance(data[key], torch.Tensor):
#             data[key] = Variable(data[key].cuda())
#     with torch.no_grad():
#         logits = model(data, "valid", acc_result)
            
        # return model.visual.cpu().numpy().flatten()
    # inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding='max_length')
    # input_ids = inputs['input_ids'].to(device)
    # attention_mask = inputs['attention_mask'].to(device)
    # token_type_ids = inputs['token_type_ids'].to(device)
    # event_type_ids = torch.zeros_like(input_ids).to(device)
    
    # input_ids = input_ids.unsqueeze(0)
    # attention_mask = attention_mask.unsqueeze(0)
    # token_type_ids = token_type_ids.unsqueeze(0)
    # event_type_ids = event_type_ids.unsqueeze(0)
    # 进行预测
    # with torch.no_grad():
    #     logits = model({"inputx": input_ids, "mask": attention_mask, "segment": token_type_ids, "event": event_type_ids}, "valid", None)
        # model.visual
    # _, outputs = results(**inputs)
    # return model.visual.cpu().numpy().flatten()

# 获取所有文本的向量表示
# vectors = np.array([get_text_embedding(text) for text in texts])

# # 应用t-SNE进行降维
# tsne = TSNE(n_components=2, random_state=42)
# X_tsne = tsne.fit_transform(vectors)

# # 可视化
# def plot_with_labels(X, labels):
#     plt.figure(figsize=(12, 8))
#     colors = ['r', 'g', 'b', 'c', 'm', 'y']
#     unique_labels = list(set(labels))
#     color_map = {label: colors[i % len(colors)] for i, label in enumerate(unique_labels)}

#     for label in unique_labels:
#         idx = [i for i, l in enumerate(labels) if l == label]
#         plt.scatter(X[idx, 0], X[idx, 1], c=color_map[label], label=label, alpha=0.5)
    
#     plt.legend()
#     plt.show()

# plot_with_labels(X_tsne, labels)